In [9]:
from langchain_elasticsearch import ElasticsearchStore
from elasticsearch import Elasticsearch

from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="nlpai-lab/KURE-v1",
    model_kwargs={"device": 'cpu'}
)

# ===== 1. Elasticsearch Client 생성 (SSL 검증 끔) =====
es_client = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "changeme"),
    verify_certs=False,
    ssl_show_warn=False
)

# ===== 2. Index & Embedding 객체 준비 =====
index_name = "cases"

elastic_vector_search = ElasticsearchStore(
    es_connection=es_client,
    index_name=index_name,
    embedding=embeddings,
)

In [10]:
def rrf_fusion(bm25_hits, knn_hits, k=60):
    scores = {}
    methods = {}  # 문서별 최종 출처 저장 (Sparse / Dense)

    # Sparse (BM25)
    for rank, (doc_id, _, method) in enumerate(bm25_hits, start=1):
        scores[doc_id] = scores.get(doc_id, 0) + 1 / (k + rank)
        # Sparse로 기록 (나중에 Dense가 덮을 수 있음)
        methods[doc_id] = method

    # Dense (KNN)
    for rank, (doc_id, _, method) in enumerate(knn_hits, start=1):
        scores[doc_id] = scores.get(doc_id, 0) + 1 / (k + rank)
        # Dense가 더 중요하다고 가정 → 동일 문서면 Dense로 덮음
        methods[doc_id] = method

    # 점수 기준 정렬
    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    # (id, score, method) 형태로 반환
    return [(doc_id, score, methods.get(doc_id)) for doc_id, score in ranked]

In [11]:
def format_results(es_client, index_name, ranked_results, top_k=5):
    formatted = []

    for rank, (doc_id, score, method) in enumerate(ranked_results[:top_k], start=1):

        # 문서 조회 (vector 제외)
        doc = es_client.get(
            index=index_name,
            id=doc_id,
            _source_excludes=["vector"]
        )

        src = doc["_source"]

        formatted.append({
            "rank": rank,
            "score": score,
            "method": method,   # Sparse or Dense
            "id": doc_id,
            "text": src.get("text", None),
            "metadata": src.get("metadata", {})
        })

    return formatted

In [12]:
query = "교사가 학생을 훈계하는 과정에서 폭행이 문제된 판례 찾아줘"

vector = embeddings.embed_query(query)

N_DOCS = 5
K = 60

bm25 = es_client.search(
    index=index_name,
    body={"query": {"match": {"text": query}}, "size": N_DOCS},
    _source_excludes=["vector"]
)["hits"]["hits"]

knn = es_client.search(
    index=index_name,
    body={
        "knn": {
            "field": "vector",
            "query_vector": vector,
            "k": N_DOCS,
            "num_candidates": N_DOCS * 10
        }
    },
    _source_excludes=["vector"]
)["hits"]["hits"]

# 방법 라벨링 적용
bm25_labeled = [(hit["_id"], hit["_score"], "Sparse") for hit in bm25]
knn_labeled = [(hit["_id"], hit["_score"], "Dense") for hit in knn]

# 🔥 RRF 수행할 때 labeled 리스트를 넣어야 함 (기존 코드 오류 수정)
results = rrf_fusion(bm25_labeled, knn_labeled, k=K)

# 문서 상세 조회
final_docs = format_results(es_client, index_name, results, top_k=5)

# ===================== 🔍 출력 ===================== #

for d in final_docs:
    print(f"[{d['rank']}위] score={d['score']:.4f} / method={d['method']}")
    print(f"판례ID: {d['metadata'].get('판례일련번호')}")
    print(f"사건명: {d['metadata'].get('사건명')}")
    print(f"선고일자: {d['metadata'].get('선고일자')}")
    print(f"참조조문: {d['metadata'].get('참조조문')}")
    print("\n--- 판례 내용 ---")
    print(d.get("text", "").strip())
    print("-"*80)

[1위] score=0.0323 / method=Dense
판례ID: 83309
사건명: 폭행·모욕
선고일자: 2004.06.10
참조조문: [1] 형법 제20조 / [2] 형법 제20조, 초·중등교육법 제18조 제1항, 제20조 제3항, 초·중등교육법시행령 제31조 제1항, 제7항 / [3] 형법 제20조

--- 판례 내용 ---
【피 고 인】 피고인 【상 고 인】 피고인 【원심판결】 대전지법 2001. 9. 20. 선고 2000노1669 판결 【주 문】 상고를 기각한다. 【이 유】 1. 상고이유 제1주장에 관하여 원심이 인용한 제1심 채용 증거들과 대조하여 보니, 여자중학교 체육교사 겸 태권도 지도교사인 피고인이 교실 밖 공개된 장소에서 피해자 공소외 1, 공소외 2를 폭행하였고 피해자 공소외 3, 공소외 4, 공소외 1에게 욕설을 하여 모욕하였다는 요지의 이 사건 공소사실들이 유죄로 증명되었다고 보아 제1심판결을 유지한 원심의 판단은 옳고 그 판단에 필요한 심리를 다하지 아니하였다거나 증거법칙에 위반하였다는 등의 잘못이 없다. 피고인이 그 공소사실과 같은 행위를 한 적이 없음에도 원심이 증거판단과 사실인정을 잘못하였다는 취지의 이 부분 주장들을 받아들이지 아니한다. 2. 상고이유 제2주장에 관하여 형법 제20조가 법령에 의한 행위 또는 업무로 인한 행위 기타 사회상규에 위반되지 아니하는 행위는 벌하지 아니한다고 규정하여 법령에 의한 학생에 대한 징계나 학생에 대한 교육적 지도행위의 경우에는 그 행위의 위법성이 조각(阻却)되는 것임은 상고이유로 주장된 바와 같다. 그런데 사회상규에 위반되지 아니하는 행위라 함은 법질서 전체의 정신이나 그의 배후에 놓여 있는 사회윤리 도의적 감정 내지 사회통념에 비추어 용인될 수 있는 행위를 말하는 것이어서 어떠한 행위가 사회상규에 위배되지 아니하는가는 구체적 사정 아래에서 합목적적, 합리적으로 고찰하여 개별적으로 판단되어야 할 것이다(대법원 2000. 4. 25. 선고 98도2389 판결 참조). 한편, 교육에 관한 중심